<a href="https://colab.research.google.com/github/Jose-Gabriel-Rodriguez/MachineLearning/blob/main/Unidad3/Practica_5_Random_Forest_Lending_Club.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://github.com/JoseGabriel-ITD/Probabilidad-y-Estadistica/blob/main/Cintilla2004.png?raw=true)

# **Machine Learning y Deep Learning**

## Unidad 3

### Practica 5: **Random Forest Prestamos Lending Club**

### Facilitador: *Dr. José Gabriel Rodríguez Rivas*

### Alumno:  _____________


## **Clasificador de bosque aleatorio**

Probaremos el método de clasificación Random Forest Classifiers con diferentes parámetros.
- El primero con hpyerparameters por defecto y
- El segundo con `class_weight='balanced'`.
- Establecemos `random_state=42` (semilla) en los clasificadores solo para garantizar la repetibilidad.

- El conjunto de datos está muy desequilibrado, con un 85 % de clase 1 (Pagado) y un 15 % de clase 0 (No pagado), podemos lograr una mejor precisión de clase 1, estableciendo un peso de clases balanceado.
- El RandomForestClassifier predeterminado no es muy sensible al cambio de class_weight, por lo que también limitamos `max_ depth` para aumentar la efectividad del cambio de class_weight.

- En esta practica, la tasa de precisión no es lo que estamos buscando. El propósito del modelo de clasificación es ayudarnos a seleccionar los préstamos que tienen más probabilidades de ser reembolsados. Por lo tanto, nuestro enfoque está en la tasa de precisión de la clase 1. Del informe de clasificación del primer modelo, podemos ver que la tasa de precisión de la clase 1 es 0,86 u 86 %.
- Esto significa que entre todos los préstamos identificados como pagados en su totalidad por el modelo de clasificación, el 86% de ellos están realmente pagados en su totalidad. Esto no es mucho mejor que la tasa de reembolso de todo el conjunto de prueba, que es de alrededor del 85 %. La razón es que el modelo solo tiene una tasa de recuperación de 0,07 en la clase 0, lo que significa que, para todos los préstamos que se cancelan, el modelo solo identifica el 7% de ellos como cancelados.





In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Importamos librerias necesarias para Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

In [ ]:
prestamos_df = pd.read_csv('prestamos_ok.csv')
prestamos_df.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,issue_year,repaid,loan_term_year,purpose_code,home_ownership_code,grade_code,addr_state_code
0,0,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,Cash,N,NaN,2011,1,3,11,4,2,2
1,1,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,Cash,N,NaN,2011,1,3,9,4,2,0
2,2,3000,3000,3000.0,36 months,18.64,109.43,E,E1,MKC Accounting,...,Cash,N,NaN,2011,1,3,0,4,4,0
3,3,5600,5600,5600.0,60 months,21.28,152.39,F,F2,NaN,...,Cash,N,NaN,2011,0,5,11,3,5,0
4,4,5375,5375,5350.0,60 months,12.69,121.45,B,B5,Starbucks,...,Cash,N,NaN,2011,0,5,9,4,1,5


## **Selección de Variables predictoras**   

**Para la seleccion de caracteristicas que se utilizarán en el modelo, la mejor opción es comprender los datos, y comprender las reglas del negocio**

Selecionamos las siguientes variables predictoras (columnas)

- **funded_amnt**             ( monto financiado)  La cantidad total comprometida con ese préstamo en ese momento.
- **loan_term_year**          ( Plazo del credito )
- **int_rate**                ( taza de interes )
- **grade_code**              ( codigo de la calificacion del crédito )
- **purpose_code**            ( codigo del proposito)
- **addr_state_code**         ( codigo de la ciudad)
- **home_ownership_code**     ( codigo situacion casa)
- **annual_inc**              ( ingresos anuales)
- **dti**                     ( Una relación calculada utilizando los pagos de deuda mensuales totales del prestatario )
- **revol_util**              ( cantidad de crédito que el prestatario está utilizando en relación con todo el crédito  )
- **pub_rec_bankruptcies**    ( Número de quiebras de registros públicos )

**Variable dependiente (variable a predecir)**
* **la etiqueta `repaid` será la elegida para el modelo de clasificación**.

**Otros aspectos a considerar**

- Solo las **características que están disponibles antes de que se inicie el préstamo pueden usarse en la clasificación**.
- Las características como **recoveries** (recuperaciones o cargo posterior a la recuperación bruta), **total_rec_prncp** (Principal recibido a la fecha), que solo están disponibles después de que se cierra el préstamo, no deben incluirse en las funciones de entrenamiento.
- Si la clasificación logra una tasa de precisión cercana al 100%, es probable que incluya características que solo estarán disponibles después de que se cierre el préstamo.



In [ ]:
X = prestamos_df[['funded_amnt', "int_rate", "grade_code", 'purpose_code', 'addr_state_code',
                 'home_ownership_code', 'annual_inc', 'dti', 'revol_util',
                 'pub_rec_bankruptcies']]

# Variable objetivo o variable a predecir
y = prestamos_df["repaid"]

### **Dividir dataset en conjunto de entrenamiento y prueba**

In [ ]:
# Dividimos el dataFrame
# stratify es para que mantenga la misma proporción de clases en ambos conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.4, stratify=y )

In [ ]:
# verificamos la cantidad de registros asignados al dataframe de entrenamiento
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11944, 10), (7964, 10), (11944,), (7964,))

## **Random Forest sin balanceo de clases**

- Construir un modelo de random forest con el atributo **class_weight** predeterminado.
- Entrenar y evaluar el modelo.

In [ ]:
# Create Regressor with default properties
rfc1 = RandomForestClassifier(random_state=23)

# Fit estimator and display score
rfc1 = rfc1.fit(X_train, y_train)

# Precisión del modelo en la fase de entrenamiento
print("Precision del clasificador en fase de entrenamiento", rfc1.score(X_train, y_train) )

Precision del clasificador en fase de entrenamiento 1.0


In [ ]:
# Realizar una prediccion con los datos de prueba
y_pred = rfc1.predict(X_test)

# Crear un informe de texto que muestre las principales métricas de clasificación.
print("\nReporte del clasificador Random Forest sin balanceo de clases : \n",
      classification_report(y_test, y_pred, target_names=["No Pagado", "Pagado"]))

print(f'\nMatriz de Confusion Random Forest sin balanceo de clases:\n', confusion_matrix(y_test, y_pred ))


Reporte del clasificador Random Forest sin balanceo de clases : 
               precision    recall  f1-score   support

   No Pagado       0.37      0.03      0.05      1177
      Pagado       0.85      0.99      0.92      6787

    accuracy                           0.85      7964
   macro avg       0.61      0.51      0.49      7964
weighted avg       0.78      0.85      0.79      7964


Matriz de Confusion Random Forest sin balanceo de clases:
 [[  34 1143]
 [  59 6728]]


### **Interpretación**  

| Métrica       | No Pagado | Pagado | Global   |
| ------------- | --------- | ------ | -------- |
| **Precision** | 0.37      | 0.85   |          |
| **Recall**    | 0.03      | 0.99   |          |
| **Accuracy**  |           |        | **0.85** |

**Conclusiones:**
* El modelo clasifica casi todo como Pagado.
* Aunque tiene alta exactitud (0.85), no detecta casi ningún préstamo “No Pagado” (recall = 0.03).
* Típico de modelos en datasets desbalanceados.


## **Random Forest con balanceo de clases**

- Construir un modelo de random forest con el atributo **class_weight** predeterminado.
- Especificar el parametro **max-depth**
- Entrenar y evaluar el modelo.

In [ ]:
rfc2 = RandomForestClassifier(class_weight='balanced', random_state=23)
rfc2 = rfc2.fit(X_train, y_train)
y_pred = rfc2.predict(X_test)

print("Reporte del clasificador con balanceo de clases: \n", classification_report(y_test, y_pred,
                                                            target_names=["No Pagado", "Pagado"] ))
print('Matriz de Confusion con balanceo de clases \n' , confusion_matrix(y_test, y_pred))


Reporte del clasificador con balanceo de clases: 
               precision    recall  f1-score   support

   No Pagado       0.30      0.01      0.02      1177
      Pagado       0.85      1.00      0.92      6787

    accuracy                           0.85      7964
   macro avg       0.58      0.50      0.47      7964
weighted avg       0.77      0.85      0.79      7964

Matriz de Confusion con balanceo de clases 
 [[  14 1163]
 [  33 6754]]


### **Interpretación**

| Métrica       | No Pagado | Pagado | Global   |
| ------------- | --------- | ------ | -------- |
| **Precision** | 0.30      | 0.85   |          |
| **Recall**    | 0.01      | 1.00   |          |
| **Accuracy**  |           |        | **0.85** |

**Conclusiones**:
* El balanceo automático no ayudó: el recall de No Pagado incluso bajó (lo ignora completamente).
* El modelo sigue clasificando casi todo como Pagado.
* Es "posible" que el desequilibrio es tan fuerte que *class_weight* no basta.
* También, es posible que la estructura del modelo (profundidad, número de árboles) no permita aprovechar el balance.

## **Parámetros ajustables en random forest**

**Parametro: n_estimators**
* Especifica el número de árboles en el bosque.
* Más árboles suelen llevar a un modelo más robusto y preciso, pero también aumentan el tiempo de entrenamiento.
* Valores comunes:
  * Usualmente se prueban valores como 100, 200, 500, y en algunos casos 1000 o más.
  * Por defecto: El valor predeterminado en scikit-learn es 100.  

**Parametro: max_depth**
- Define la profundidad máxima de cada árbol.
- Limitar la profundidad ayuda a controlar el sobreajuste.
- Si no se especifica (None), cada árbol crecerá hasta que todas las hojas sean puras o contengan menos muestras que min_samples_split.
- Valores típicos son: 5, 10, 20, dependiendo del tamaño y complejidad de los datos.

## **Modelo 3 con n_estimators=200 y max_depth=7**

In [ ]:
rfc3 = RandomForestClassifier(n_estimators=200, max_depth=7, class_weight='balanced', random_state=23)

# Fit estimator and display score
rfc3 = rfc3.fit(X_train, y_train)

y_pred = rfc3.predict(X_test)

print("Reporte del clasificador con balanceo de clases n_estimators=200, max_depth=7: \n",
                   classification_report(y_test, y_pred, target_names=["No Pagado", "Pagado"]))
print('Matriz de Confusion con balanceo de clases n_estimators=200, max_depth=7\n' , confusion_matrix(y_test, y_pred))

Reporte del clasificador con balanceo de clases n_estimators=200, max_depth=7: 
               precision    recall  f1-score   support

   No Pagado       0.23      0.57      0.32      1177
      Pagado       0.90      0.66      0.76      6787

    accuracy                           0.65      7964
   macro avg       0.56      0.62      0.54      7964
weighted avg       0.80      0.65      0.70      7964

Matriz de Confusion con balanceo de clases n_estimators=200, max_depth=7
 [[ 670  507]
 [2293 4494]]


### **Interpretación**  

| Clase               | Precision | Recall | f1-score |
| ------------------- | --------- | ------ | -------- |
| **No Pagado**       | 0.23      | 0.57   | 0.32    |
| **Pagado**          | 0.90      | 0.66   | 0.766     |
| **Accuracy global** |           |        | **0.65** |  


**Conclusiones**
* Este modelo mejora drásticamente la detección de préstamos “No Pagados” (recall = 0.57).
* Logra un balance real entre ambas clases, sacrificando algo de exactitud global.
* En un escenario crediticio, este es el comportamiento más valioso, ya que **es preferible detectar morosos aunque haya falsos positivos**.

### **Modelo 4 con n_estimators=200 y max_depth=10**

In [ ]:
rfc4 = RandomForestClassifier(n_estimators=200, max_depth=10, class_weight='balanced', random_state=23)

# Fit estimator and display score
rfc4 = rfc4.fit(X_train, y_train)

y_pred = rfc4.predict(X_test)

print("Reporte del clasificador con balanceo de clases n_estimators=200, max_depth=10: \n",
                      classification_report(y_test, y_pred, target_names=["No Pagado", "Pagado"]))
print('Matriz de Confusion con balanceo de clases n_estimators=200, max_depth=10\n' , confusion_matrix(y_test, y_pred))

Reporte del clasificador con balanceo de clases n_estimators=200, max_depth=10: 
               precision    recall  f1-score   support

   No Pagado       0.26      0.39      0.31      1177
      Pagado       0.88      0.80      0.84      6787

    accuracy                           0.74      7964
   macro avg       0.57      0.60      0.58      7964
weighted avg       0.79      0.74      0.76      7964

Matriz de Confusion con balanceo de clases n_estimators=200, max_depth=10
 [[ 458  719]
 [1325 5462]]


### **Interpretación**  

| Clase               | Precision | Recall | f1-score |
| ------------------- | --------- | ------ | -------- |
| **No Pagado**       | 0.26      | 0.39   | 0.31     |
| **Pagado**          | 0.88      | 0.80   | 0.84     |
| **Accuracy global** |           |        | **0.74** |  


**Conclusiones**
* Recupera algo de exactitud (0.74), pero pierde sensibilidad para “No Pagado” (recall baja a 0.39).
* Aumentar la profundidad hace que el modelo se incline otra vez hacia la clase mayoritaria.
* Aun así, mantiene un rendimiento más equilibrado que los modelos sin balanceo.

### **Comparativa general de los 4 modelos**

| Modelo           | Accuracy | Recall (No Pagado) | Recall (Pagado) | Comentario                                      |
| ---------------- | -------- | ------------------ | --------------- | ----------------------------------------------- |
| RF sin balanceo  | 0.85     | 0.03               | 0.99            | Muy sesgado, casi no detecta morosos            |
| RF balanceado    | 0.85     | 0.01               | 1.00            | Sin mejora, ignora clase minoritaria            |
| RF bal. depth=7  | 0.65     | **0.57**           | 0.66            | Mejor equilibrio, detecta la mayoría de morosos |
| RF bal. depth=10 | 0.74     | 0.39               | 0.80            | Buen compromiso entre ambas clases              |  

**Conclusiones**
* El mejor modelo sigue siendo el Random Forest balanceado con max_depth=7 y n_estimators=200
* Es el único que detecta correctamente más de la mitad de los préstamos impagos.
* Aunque la exactitud total (0.65) es menor, su recall de la clase crítica (No Pagado) es el más alto.
* En problemas de riesgo financiero, minimizar los falsos negativos (morosos no detectados) es más importante que tener alta precisión global.


## **Comparativa de Modelos con Gráfica de radar**

In [ ]:
# ==========================================
# Gráfica de radar - Random Forest (Plotly)
# ==========================================
import plotly.graph_objects as go

# --- Nombres de los modelos ---
modelos = [
    "RF sin balanceo",
    "RF balanceado",
    "RF bal. n=200, depth=7",
    "RF bal. n=200, depth=10"
]

# --- Métricas reales ---
accuracy = [0.85, 0.85, 0.65, 0.73]
recall_no_pagado = [0.03, 0.01, 0.58, 0.40]
recall_pagado = [0.99, 1.00, 0.66, 0.78]
f1_macro = [0.49, 0.47, 0.54, 0.57]

# --- Ejes ---
metricas = ["Accuracy", "Recall No Pagado", "Recall Pagado", "F1 Macro"]

# --- Crear la figura ---
fig = go.Figure()

# Agregar cada modelo al radar
for i, modelo in enumerate(modelos):
    fig.add_trace(go.Scatterpolar(
        r=[accuracy[i], recall_no_pagado[i], recall_pagado[i], f1_macro[i], accuracy[i]],
        theta=metricas + [metricas[0]],
        fill='toself',
        name=modelo
    ))

# --- Personalización ---
fig.update_layout(
    title="Comparación de modelos Random Forest (Radar Plot)",
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1], tickvals=[0.2, 0.4, 0.6, 0.8, 1.0]),
    ),
    showlegend=True
)
fig.update_layout(width=800, height=600)
fig.show()


### **Interpretación**

* Cuanto más grande y uniforme sea el polígono → mejor desempeño global del modelo.
* Los modelos “sin balanceo” tienen alta precisión para Pagado, pero casi ignoran No Pagado (recall muy bajo).
* Los modelos balanceados con ajuste de profundidad (7 o 10) mejoran notablemente el recall de “No Pagado”, aunque bajan un poco en accuracy.
* Esto muestra la compensación entre balance de clases y precisión general.

## **Busqueda de parámetros optimos**

**¿Cómo funciona GridSearchCV?**
* Necesita un estimador (p. ej. RandomForestClassifier()),
* **un espacio de búsqueda (param_grid**: diccionario con listas de valores por parámetro),
* **una métrica de evaluación** (scoring),
* un esquema de validación (cv, p. ej. StratifiedKFold),
* y opciones como n_jobs=-1, refit=True, etc.  

**Qué hace:**
* Arma el **producto cartesiano de combinaciones** en param_grid,
para cada combinación, realiza validación cruzada,
* **Calcula la(s) métrica(s), y elige la mejor según scoring**.

**Qué devuelve:**
* **best_params_, best_score_, best_estimator_,**
cv_results_ con todo el detalle por combinación,
* si refit=True (por defecto), reentrena el mejor modelo con todo el conjunto de entrenamiento.



## **NOTA: EL SIGUIENTE CODIGO TARDA EN EJECUTARSE DE 20 A 45 MINUTOS.**

**Considerar que:**
* **“5 folds”:** se está usando validación cruzada con 5 particiones (k-fold CV). Cada combinación de hiperparámetros se evalúa en 5 divisiones del conjunto de entrenamiento.
* **“162 candidates”:** el grid que definiste tiene 162 combinaciones posibles de parámetros (producto cartesiano de todas las listas en param_grid).
* **“totalling 810 fits”:** se entrenarán **810** modelos en total, porque: **número de combinaciones *X* número de folds = 162 *5=810 combinaciones**


In [ ]:
# Búsqueda de hiperparámetros con GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Iniciamos con un Modelo base con balanceo de clases (para compensar desbalance)
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# --- Definimos el espacio de búsqueda ---
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 10 ],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# --- Configuración de GridSearchCV ---
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='f1_macro',   # Se puede cambiar a 'recall_macro', 'accuracy', etc.
    cv=3,                 # 3 particiones para validación cruzada
    n_jobs=-1,            # usa todos los núcleos disponibles
    verbose=2             # para ver el progreso
)

# --- Ejecutar búsqueda ---
grid_search.fit(X_train, y_train)

# --- Mostrar los mejores parámetros ---
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

print("\n Mejor puntaje promedio (validación cruzada):")
print(grid_search.best_score_)

# --- Entrenar modelo final con los mejores parámetros ---
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

# --- Evaluar en conjunto de prueba ---
print("\n Reporte de Clasificación (modelo óptimo):")
print(classification_report(y_test, y_pred_best, target_names=["No Pagado", "Pagado"]))

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_best))


Fitting 5 folds for each of 162 candidates, totalling 810 fits
Mejores hiperparámetros encontrados:
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}

 Mejor puntaje promedio (validación cruzada):
0.575194985637021

 Reporte de Clasificación (modelo óptimo):
              precision    recall  f1-score   support

   No Pagado       0.25      0.41      0.31      1177
      Pagado       0.89      0.79      0.83      6787

    accuracy                           0.73      7964
   macro avg       0.57      0.60      0.57      7964
weighted avg       0.79      0.73      0.76      7964

Matriz de Confusión:
[[ 488  689]
 [1439 5348]]


### **Interpretacion**

* Aunque el **GridSearchCV encontró un conjunto de hiperparámetros que maximizan el promedio de desempeño global** (macro o accuracy), eso **NO necesariamente significa que priorice la detección de la clase minoritaria** (“No Pagado”), que es justamente la más importante en un escenario de riesgo crediticio.
* El proceso busca maximizar el rendimiento global del modelo, no de una clase específica.
* Esto hace que **el algoritmo tienda a favorecer la clase mayoritaria (“Pagado”)**, ya que predecirla correctamente genera una aparente mejora en accuracy, **aunque no sea útil para el objetivo real (detectar impagos)**.

### **Cómo hacer que GridSearchCV priorice la clase minoritaria**

1. **Usar un “scoring”** enfocado en la clase minoritaria.
* Esto hará que GridSearch seleccione el modelo que maximiza el recall de la clase 0, es decir, que detecte más impagos, incluso si eso sacrifica algo de precisión global.
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score

**Ejemplo: priorizar recall de la clase minoritaria (detecta más impagos)**
 ```python
scorer = make_scorer(recall_score, pos_label=0)

grid_search = GridSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state=23),
    param_grid=param_grid,
    scoring=scorer,  # << prioriza recall de la clase No Pagado
    cv=5,
    n_jobs=-1,
    verbose=2
)
```

2. **Usar un “scoring” balanceado: F2-score o weighted F1**
* Otra opción más equilibrada es dar mayor peso al recall, pero sin ignorar la precisión.
* Puedes crear un F2-score (más sensible al recall) personalizado:  
* Esto le dice, prefiero que detectes más impagos, incluso si cometes más falsos positivos

 ```python
from sklearn.metrics import make_scorer, fbeta_score

# F2 da más peso al recall (importante para detectar impagos)
scorer_f2 = make_scorer(fbeta_score, beta=2, pos_label=0)

grid_search = GridSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state=23),
    param_grid=param_grid,
    scoring=scorer_f2,
    cv=5,
    n_jobs=-1,
    verbose=2
)

 ```

3. **Usar múltiples métricas y evaluar todas**
* También puedes pedirle a GridSearchCV que calcule varias métricas y tú eliges cuál priorizar.
* De este modo, GridSearch almacenará todas las métricas para cada combinación de hiperparámetros, pero solo reentrenará el mejor modelo según la que tú definas (por ejemplo, recall_no_pagado).

```python
grid_search = GridSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state=23),
    param_grid=param_grid,
    scoring={
        'recall_no_pagado': make_scorer(recall_score, pos_label=0),
        'f1_no_pagado': make_scorer(f1_score, pos_label=0),
        'accuracy': 'accuracy'
    },
    refit='recall_no_pagado',  # <--- elige el modelo que maximice recall de No Pagado
    cv=5,
    n_jobs=-1,
    verbose=2
)

```









## **Busqueda de parámetros optimos priorizando detección de NO pagados**

### **NOTA: La ejecucion puede tardar de 20 a 45 minutos**

In [ ]:
# Optimización de Random Forest priorizando detección de impagos

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score, classification_report, confusion_matrix

# Definición del modelo base con balanceo de clases
# El parámetro class_weight='balanced' ajusta el peso inversamente proporcional
# a la frecuencia de cada clase, ayudando a tratar el desbalance.

modelo_base = RandomForestClassifier(
    class_weight='balanced',
    random_state=23
)

# Definición de la cuadrícula de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Definición del criterio de evaluación
# Se usará el recall de la clase "No Pagado" (pos_label=0)
# Esto hace que el modelo busque detectar el mayor número posible de impagos.

scorer = make_scorer(recall_score, pos_label=0)

# Configuración del GridSearchCV
grid_search = GridSearchCV(
    estimator=modelo_base,
    param_grid=param_grid,
    scoring=scorer,       # prioriza el recall de la clase No Pagado
    cv=5,                 # validación cruzada con 5 particiones
    n_jobs=-1,            # usa todos los núcleos disponibles
    verbose=2
)

# Entrenamiento del modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

print("\nBúsqueda finalizada.")
print(f"Mejores hiperparámetros encontrados:\n{grid_search.best_params_}")

# Evaluación del mejor modelo
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

print("\nReporte de Clasificación (modelo optimizado para detectar impagos):\n")
print(classification_report(y_test, y_pred))

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))


Fitting 5 folds for each of 162 candidates, totalling 810 fits

Búsqueda finalizada.
Mejores hiperparámetros encontrados:
{'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}

Reporte de Clasificación (modelo optimizado para detectar impagos):

              precision    recall  f1-score   support

           0       0.22      0.66      0.33      1177
           1       0.91      0.60      0.72      6787

    accuracy                           0.61      7964
   macro avg       0.57      0.63      0.53      7964
weighted avg       0.81      0.61      0.66      7964

Matriz de Confusión:
[[ 774  403]
 [2727 4060]]



**Con los Mejores hiperparámetros encontrados; el modelo:**
* **Tiene árboles relativamente poco profundos (max_depth=5)** → más generalización, menos sobreajuste.
* **Usa 200 árboles** → buena estabilidad estadística.
* **max_features='sqrt'** → selecciona aleatoriamente √(n_features) en cada división, lo que mejora la diversidad de los árboles.
* **min_samples_split=10** evita divisiones innecesarias.

**Reporte de clasificación**

| Clase                      | Precision | Recall   | F1-score | Soporte | Interpretación                                                                                                      |
| -------------------------- | --------- | -------- | -------- | ------- | ------------------------------------------------------------------------------------------------------------------- |
| **0 = No Pagado (impago)** | **0.22**  | **0.66** | **0.33** | 1177    | El modelo logra detectar **66%** de los impagos — una **gran mejora** respecto a KNN (que apenas llegaba al 10%) |
| **1 = Pagado**             | 0.91      | 0.60     | 0.72     | 6787    | Disminuye el recall de Pagado (muchos falsos negativos), pero esto **es aceptable en contexto financiero**          |  
  
**Matriz de confusión**  
| Tipo          | Valor                                                          | Interpretación |
| ------------- | -------------------------------------------------------------- | -------------- |
| **TP (774)**  | Impagos correctamente identificados                            |                |
| **FN (403)**  | Impagos clasificados como pagados (riesgo no detectado)        |                |
| **FP (2727)** | Pagados clasificados como impagos (clientes buenos rechazados) |                |
| **TN (4060)** | Pagados correctamente clasificados                             |                |



